In [1]:
import nltk
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sb
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")
import unidecode
#from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.stem import PorterStemmer
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import matplotlib.animation as animation
import operator
#import plotly.express as px
from collections import Counter
%matplotlib inline
import pandas_profiling

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
data=pd.read_csv("val.csv")
data=data.drop('tokenized',axis=1)
#data=data.drop('Unnamed: 0',axis=1)
data.head()

,Unnamed: 0,text,sentiment
0,0,oh marly im sorry hope find soon,neutral
1,1,play ghost onlin realli interesting new updat ...,positive
2,2,clean hous famili com later today,neutral
3,3,gotta restart comput thought win suppos put e...,neutral
4,4,see wat mean bout follw friidays it call lose ...,neutral


In [3]:

data.dropna
data.describe()

,Unnamed: 0
count,27447.000000
mean,13723.989288
std,7923.429111
min,0.000000
25%,6862.500000
50%,13724.000000
75%,20585.500000
max,27447.000000


In [4]:
data['text'] = data['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
data['text'].head()

0                     oh marly im sorry hope find soon
1    play ghost onlin realli interesting new updat ...
2                    clean hous famili com later today
3    gotta restart comput thought win suppos put en...
4    see wat mean bout follw friidays it call lose ...
Name: text, dtype: object

In [5]:
data['text'] = data['text'].apply(lambda x : ' '.join([word for word in str(x).split() if not word in set(stopwords.words('english'))]))

In [6]:
lemmatizer = WordNetLemmatizer()
data['text'] = data['text'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in str(x).split()]))

In [7]:
from textblob import TextBlob
data['text'][:5].apply(lambda x: str(TextBlob(x).correct()))

0                      oh mary in sorry hope find soon
1    play ghost online really interesting new updat...
2                   clean house family com later today
3    gutta start compute thought win suppose put en...
4    see wat mean bout follow friday call lose flow...
Name: text, dtype: object

In [8]:
def clean_tweets(tweet):
    # remove special characters 
    tweet = re.sub('[^ a-zA-Z0-9]', '', tweet)
    # remove Numbers
    tweet = re.sub('[0-9]', '', tweet)
    return tweet
data['text'] = data['text'].apply(clean_tweets)

In [9]:
def tokenize(text):
    return word_tokenize(text)
data['tokenized'] = data['text'].apply(tokenize)
data.head()

,Unnamed: 0,text,sentiment,tokenized
0,0,oh marly im sorry hope find soon,neutral,"[oh, marly, im, sorry, hope, find, soon]"
1,1,play ghost onlin realli interesting new updat ...,positive,"[play, ghost, onlin, realli, interesting, new,..."
2,2,clean hous famili com later today,neutral,"[clean, hous, famili, com, later, today]"
3,3,gotta restart comput thought win suppos put en...,neutral,"[got, ta, restart, comput, thought, win, suppo..."
4,4,see wat mean bout follw friidays call lose fll...,neutral,"[see, wat, mean, bout, follw, friidays, call, ..."


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
cv = TfidfVectorizer(max_features=5000) 
x = cv.fit_transform(data['text'].tolist()).toarray()
#y=pd.get_dummies(df['sentiment'])
#y=y.iloc[:,1].values
#x_train, x_test, y_train, y_test = train_test_split(x, data['sentiment'], test_size = 0.20, random_state = 123)

In [11]:
data.sentiment[data.sentiment == 'positive'] = 1
data.sentiment[data.sentiment == 'neutral'] = 0
data.sentiment[data.sentiment == 'negative'] = 2
y=data['sentiment'].astype('float')
print(y)

0        0.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
27442    2.0
27443    2.0
27444    1.0
27445    0.0
27446    1.0
Name: sentiment, Length: 27447, dtype: float64


In [12]:
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import expon,uniform,randint

#Sklearn imports
from sklearn import linear_model
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score,cross_val_predict,validation_curve
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [13]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0) #Basic train_test_split works here
y_test.shape

(6862,)

In [15]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')

In [ ]:
svc.fit(x_train,y_train)

In [ ]:
y_pred=svc.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [14]:
from sklearn.ensemble import RandomForestClassifier
m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, n_jobs=-1)
m.fit(x_train, y_train)
predictions = m.predict(x_test)
print(classification_report(y_test, predictions))
#print("Accuracy :",accuracy_score(y_test, predictions),'\n')

              precision    recall  f1-score   support

         0.0       0.65      0.72      0.69      2774
         1.0       0.75      0.76      0.75      2155
         2.0       0.72      0.59      0.65      1933

    accuracy                           0.70      6862
   macro avg       0.71      0.69      0.70      6862
weighted avg       0.70      0.70      0.70      6862

Accuracy : 0.6981929466627805 



In [16]:
# With single train test split
clf = LogisticRegression(solver='liblinear').fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.6926552025648499
[[2122  330  322]
 [ 560 1502   93]
 [ 704  100 1129]]
              precision    recall  f1-score   support

         0.0       0.63      0.76      0.69      2774
         1.0       0.78      0.70      0.74      2155
         2.0       0.73      0.58      0.65      1933

    accuracy                           0.69      6862
   macro avg       0.71      0.68      0.69      6862
weighted avg       0.70      0.69      0.69      6862



In [19]:
## Multinomial NB with single train test split
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB().fit(x_train, y_train)

y_pred=nb_model.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.6394637132031478
[[2144  356  274]
 [ 784 1294   77]
 [ 878  105  950]]
              precision    recall  f1-score   support

         0.0       0.56      0.77      0.65      2774
         1.0       0.74      0.60      0.66      2155
         2.0       0.73      0.49      0.59      1933

    accuracy                           0.64      6862
   macro avg       0.68      0.62      0.63      6862
weighted avg       0.66      0.64      0.64      6862

